<a href="https://colab.research.google.com/github/jimregan/wolnelektury-audio-corpus/blob/new-vocab/MLM_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/awslabs/mlm-scoring

In [2]:
from transformers import *
model = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
tokenizer = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
nlp = pipeline('fill-mask', model=model, tokenizer=tokenizer)
vocab = None

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [4]:
!pip install mxnet-cu101mkl

     |████████████████████████████████| 712.3MB 25kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [5]:
from mlm.scorers import MLMScorer, MLMScorerPT
from mlm.models import get_pretrained
import mxnet as mx
ctxs = [mx.gpu()]
scorer = MLMScorerPT(model, vocab, tokenizer, ctxs)

/usr/local/lib/python3.6/dist-packages/mxnet/optimizer/optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))


In [ ]:
!head -n 440 mlm-scoring/examples/asr-librispeech-espnet/data/test-clean.am.json

In [7]:
!wget http://poleval.pl/task1/Poleval2020Task1Eval.tar.xz

--2020-08-20 07:40:32--  http://poleval.pl/task1/Poleval2020Task1Eval.tar.xz
Resolving poleval.pl (poleval.pl)... 213.135.36.94
Connecting to poleval.pl (poleval.pl)|213.135.36.94|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12592244 (12M) [application/x-tar]
Saving to: ‘Poleval2020Task1Eval.tar.xz’

Poleval2020Task1Eva 100%[===================>]  12.01M  2.03MB/s    in 6.9s    

2020-08-20 07:40:41 (1.74 MB/s) - ‘Poleval2020Task1Eval.tar.xz’ saved [12592244/12592244]



In [8]:
!tar xf Poleval2020Task1Eval.tar.xz

In [ ]:
import gc
import torch
out = open("/content/scored", "a")
with open(file="/content/Poleval2020Task1Eval/nbest.txt", mode="r", encoding="utf-8") as nbest:
  for line in nbest:
    sentence = []
    words = []
    score = []
    words = line.strip().split(' ')
    id = words[0]
    sentence = ' '.join(words[1:])
    score = scorer.score_sentences([sentence])
    out.write("%s %f\n" % (id, score[0]))
    torch.cuda.empty_cache()
    gc.collect()
